In [0]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df = pd.read_csv("/content/gdrive/My Drive/textract_train.csv")

In [0]:
sentences = df['Contents']
labels = df['Label']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.2, random_state=42, stratify = labels)

In [0]:
   
def is_balanced(s):
    stack = []
    for ch in s:
        if ch == '(':
            stack.append(0)
        elif ch == '[':
            stack.append(1)
        elif ch == '{':
            stack.append(2)
        elif ch == ')':
            if not stack:
                return False
            if stack[-1] != 0:
                return False
            stack.pop()
        elif ch == ']':
            if not stack:
                return False
            if stack[-1] != 1:
                return False
            stack.pop()
        elif ch == '}':
            if not stack:
                return False
            if stack[-1] != 2:
                return False
            stack.pop()
    return not stack

In [0]:
mst_train = []
mst_test = []
mst2_train = []
mst3_train = []
mst4_train = []
mst5_train = []
mst6_train = []
mst7_train = []
mst8_train = []

mst2_test = []
mst3_test = []
mst4_test = []
mst5_test = []
mst6_test = []
mst7_test = []
mst8_test = []
p9=re.compile(r" ve[^A-z]")
import re
def counting_Mistakes2(text):  
    return len(re.findall(r"\.[^\s-]", text))
def counting_Mistakes1(text):
    return len(re.findall(r",[^\s-]", text))
def counting_Mistakes3(text):
    return text.count(' .')
def counting_Mistakes4(text):
    return len(re.findall(r" s[^A-z]", text))
def counting_Mistakes5(text):
    text_bracket_only = list(filter(lambda x: x in "(){}[]", text))
    return 0 if is_balanced(text_bracket_only) else 1
def counting_Mistakes6(text):
    return text.count(',,')
def counting_Mistakes8(text):
    return text.count(' :')
def counting_Mistakes9(text):
    return len(p9.findall(text))

for sentence in X_train:
  mst_train.append(counting_Mistakes2(sentence))
  mst2_train.append(counting_Mistakes1(sentence))
  mst3_train.append(counting_Mistakes3(sentence))
  mst4_train.append(counting_Mistakes4(sentence))
  mst5_train.append(counting_Mistakes5(sentence))
  mst6_train.append(counting_Mistakes6(sentence))
  mst7_train.append(counting_Mistakes8(sentence))
  mst8_train.append(counting_Mistakes9(sentence))


for sentence in X_test:
  mst_test.append(counting_Mistakes2(sentence))
  mst2_test.append(counting_Mistakes1(sentence))
  mst3_test.append(counting_Mistakes3(sentence))
  mst4_test.append(counting_Mistakes4(sentence))
  mst5_test.append(counting_Mistakes5(sentence))
  mst6_test.append(counting_Mistakes6(sentence))
  mst7_test.append(counting_Mistakes8(sentence))
  mst8_test.append(counting_Mistakes9(sentence))


In [0]:
def clean_word(sample):
    to_replace = """[,\.!?'"123456789]-():;"""
    for char in to_replace:
        sample = sample.replace(char,'')
    return sample

In [9]:
new_train = []
for article in tqdm(X_train):
    new_article = []
    for word in article.split(' '):
        new_word = clean_word(word)
        if new_word != '' and new_word != ' ':
            new_article.append(new_word)
    new_train.append(new_article)

100%|██████████| 7200/7200 [00:16<00:00, 446.06it/s]


In [10]:
new_test = []
for article in tqdm(X_test):
    new_article = []
    for word in article.split(' '):
        new_word = clean_word(word)
        if new_word != '' and new_word != ' ':
            new_article.append(new_word)
    new_test.append(new_article)

100%|██████████| 1800/1800 [00:03<00:00, 450.33it/s]


In [11]:
from gensim.models import Word2Vec, KeyedVectors
w2v_size = 300
w2v_model = Word2Vec(min_count=100,
                     window=2,
                     size=w2v_size,
                     workers=2)
w2v_model.build_vocab(new_train)
w2v_model.train(new_train, total_examples=w2v_model.corpus_count, epochs=30)
#w2v_model = Word2Vec.load("GoogleNews-vectors-negative300.bin.gz")
#w2v_model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

(94185646, 148267200)

In [0]:
# vectorizing train data
X_train = new_train
X_dev = new_test
padding= np.zeros(w2v_size)
X_train_vec = []
dim = 512
for sample in X_train:
    vec = []
    k = 0
    for word in sample:
        if k >= dim:
            break
        k += 1
        try:
            vec.append(w2v_model.wv[word])
        except:
            vec.append(padding)
    while k < dim:
        vec.append(padding)
        k += 1
    X_train_vec.append(np.array(vec))
X_train_vec = np.array(X_train_vec)

# vectorizing dev data
padding= np.zeros(w2v_size)
X_dev_vec = []
for sample in X_dev:
    vec = []
    k = 0
    for word in sample:
        if k >= dim:
            break
        k += 1
        try:
            vec.append(w2v_model.wv[word])
        except:
            vec.append(padding)
    while k < dim:
        vec.append(padding)
        k += 1
    X_dev_vec.append(np.array(vec))
X_dev_vec = np.array(X_dev_vec)

In [13]:
X_dev = X_dev_vec
X_train = X_train_vec
y_dev = y_test
y_train = np.array(list(y_train))
y_dev = np.array(list(y_dev))
mst_train = np.array(list(mst_train))
mst_test = np.array(list(mst_test))
mst2_train = np.array(list(mst2_train))
mst3_train = np.array(list(mst3_train))
mst4_train = np.array(list(mst4_train))
mst5_train = np.array(list(mst5_train))
mst6_train = np.array(list(mst6_train))
mst7_train = np.array(list(mst7_train))
mst8_train = np.array(list(mst8_train))

mst2_test = np.array(list(mst2_test))
mst3_test = np.array(list(mst3_test))
mst4_test = np.array(list(mst4_test))
mst5_test = np.array(list(mst5_test))
mst6_test = np.array(list(mst6_test))
mst7_test = np.array(list(mst7_test))
mst8_test = np.array(list(mst8_test))


print(X_train.shape)
print(X_dev.shape)
print(y_train.shape)
print(y_dev.shape)
print(mst_train.shape)
print(mst_test.shape)

(7200, 512, 300)
(1800, 512, 300)
(7200,)
(1800,)
(7200,)
(1800,)


In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, LSTM, Bidirectional, GlobalMaxPooling1D, Conv1D, Dropout, MaxPool1D,Input,ThresholdedReLU, CuDNNLSTM
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, Callback, EarlyStopping

Using TensorFlow backend.


In [36]:
inputs = Input(shape=(512,300))

x = Conv1D(128, 3, padding='same')(inputs)
x = ThresholdedReLU(theta=0.75)(x)
x = Conv1D(256, 5, padding = 'same')(x)
x = ThresholdedReLU(theta=0.75)(x)
x = GlobalMaxPooling1D()(x)
#x = Dropout(0.5)(x)
# x = Bidirectional(CuDNNLSTM(128, return_sequences = True))(inputs)
# x = Dropout(0.25)(x)
# x = Bidirectional(CuDNNLSTM(64))(x)
# x = Dropout(0.25)(x)

# x = Conv1D(64, 3, 1, activation = 'relu')(inputs)
# x = Conv1D(128, 3, 1, activation = 'relu')(x)
# x = CuDNNLSTM(32, return_sequences = True)(x)
# x = CuDNNLSTM(32, return_sequences = True)(x)
# x = CuDNNLSTM(32)(x)
conv_output = Dense(64, activation = 'relu')(x)

seq_lenght_input = Input(shape = (1,), name = 'seq_lenght_input')
ms2 = Input(shape = (1,), name = 'ms2_imput')
ms3 = Input(shape = (1,), name = 'ms3_input')
ms4 = Input(shape = (1,), name = 'ms4_input')
ms5 = Input(shape = (1,), name = 'ms5_input')
ms6 = Input(shape = (1,), name = 'ms6_input')
ms7 = Input(shape = (1,), name = 'ms7_input')
ms8 = Input(shape = (1,), name = 'ms8_input')

x = keras.layers.concatenate([seq_lenght_input, ms2, ms3, ms4, ms5, ms6, ms7, ms8])
ot = Dense(64,activation = 'relu')(x)
x = keras.layers.concatenate([ot,conv_output])

#x = conv_output
#x = Dense(64,activation = 'relu')(x)
x = Dense(32,activation = 'relu')(x)
x = Dense(16,activation = 'relu')(x)

#x = Dropout(0.25)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[inputs, seq_lenght_input, ms2, ms3, ms4, ms5, ms6, ms7, ms8] , outputs=predictions)

model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:

mcp_save = ModelCheckpoint('tempMdlml.hdf5', save_best_only=True, monitor='val_loss', mode='min')
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
csv_logger = CSVLogger('log2.csv', append=True, separator=';')

In [38]:
model.fit([X_train, mst_train,mst2_train,mst3_train,mst4_train, mst5_train, mst6_train, mst7_train, mst8_train],y_train, epochs = 2000, 
          validation_data=([X_dev, mst_test, mst2_test, mst3_test, mst4_test, mst5_test, mst6_test, mst7_test, mst8_test],y_dev), 
          batch_size = 32,callbacks= [mcp_save, earlyStopping, csv_logger])

Train on 7200 samples, validate on 1800 samples
Epoch 1/2000
7200/7200 [==============================] - 41s 6ms/sample - loss: 0.4449 - acc: 0.8076 - val_loss: 0.4006 - val_acc: 0.8222
Epoch 2/2000
7200/7200 [==============================] - 29s 4ms/sample - loss: 0.3728 - acc: 0.8314 - val_loss: 0.4017 - val_acc: 0.8239
Epoch 3/2000
7200/7200 [==============================] - 29s 4ms/sample - loss: 0.3455 - acc: 0.8396 - val_loss: 0.3927 - val_acc: 0.8267
Epoch 4/2000
7200/7200 [==============================] - 29s 4ms/sample - loss: 0.2993 - acc: 0.8625 - val_loss: 0.4221 - val_acc: 0.8067
Epoch 5/2000
7200/7200 [==============================] - 29s 4ms/sample - loss: 0.2279 - acc: 0.9024 - val_loss: 0.4949 - val_acc: 0.8056
Epoch 6/2000
7200/7200 [==============================] - 29s 4ms/sample - loss: 0.1524 - acc: 0.9365 - val_loss: 0.6033 - val_acc: 0.8028
Epoch 7/2000
7200/7200 [==============================] - 29s 4ms/sample - loss: 0.1020 - acc: 0.9610 - val_loss: 0.88

In [0]:
!ls